In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('output\listing_details_cleaned.csv')
data

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Kích thước chiều dài (m),Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng
0,Khánh Hòa,Nha Trang,Phường Phương Sơn,Đường Hải Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2.800000e+09,...,9.0,1,Nở hậu,3.0,5.0,Đất ở,"Nhà gồm 1 trệt + 1 lửng + 1 lầu, thiết kế 4 ph...",12.252321,109.178597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
1,Hà Nội,Nam Từ Liêm,Phường Tây Mỗ,Đường Tây Mỗ,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,5.700000e+09,...,NaN,1,Chữ nhật,5.0,6.0,Đất ở,Nhà Diện tích 52m rộng rãi thoáng mát ở được g...,21.004792,105.749786,"[""https://file4.batdongsan.com.vn/crop/600x315..."
2,Hồ Chí Minh,Phú Nhuận,Phường 5,Đường Thích Quảng Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,1.050000e+10,...,34.0,1,Chữ nhật,3.5,12.0,Đất ở,| HẺM Ô TÔ 7 CHỖ - 64M2 - ĐÃ XÂY FULL KHÔNG Q...,10.805619,106.682205,"[""https://file4.batdongsan.com.vn/crop/600x315..."
3,Hà Nội,Cầu Giấy,Phường Quan Hoa,Đường Nguyễn Khánh Toàn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,1.820000e+10,...,NaN,1,Chữ nhật,3.5,6.0,Đất ở,"Bán nhà Phố Nguyễn Khánh Toàn, Cầu Giấy, khu p...",21.037317,105.800733,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4,Hồ Chí Minh,Bình Tân,Phường Tân Tạo,Đường Lê Đình Cẩn,302/,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,NaN,...,12.0,1,Nở hậu,5.0,12.0,Đất ở,Diện tích 62m² - hẻm 6m thông - ngang 5.13 dài...,10.759390,106.599232,"[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18907,Hồ Chí Minh,Gò Vấp,Phường 16,Đường Số,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,05/07/2025,NaN,8.700000e+09,...,NaN,1,Chữ nhật,5.0,11.0,Đất ở,"Nhà riêng 4 tầng tại Đường Số 9, Phường 16, Gò...",10.844344,106.668257,"[""https://file4.batdongsan.com.vn/crop/600x315..."
18908,Hồ Chí Minh,Quận 10,Phường 12,Đường Hòa Hưng,91,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,05/07/2025,NaN,8.100000e+10,...,30.0,1,Chữ nhật,2.0,15.0,Đất ở,Bán nhà hẻm nhựa 91 Hòa Hưng P12 Q10. | - Vị t...,10.778993,106.673454,"[""https://file4.batdongsan.com.vn/crop/600x315..."
18909,Hồ Chí Minh,Tân Phú,Phường Tân Quý,NaN,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-phuong...,Rao bán,01/07/2025,NaN,9.900000e+09,...,543.0,1,Nở hậu,2.0,0.0,Đất ở,"Bán nhà mặt tiền nội bộ P. Tân Quý, Q. Tân Phú...",10.793344,106.624982,"[""https://file4.batdongsan.com.vn/crop/600x315..."
18910,Hồ Chí Minh,Thủ Đức,Phường Bình Chiểu,Đường Ngô Chí Quốc,201,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,28/06/2025,NaN,4.500000e+09,...,NaN,1,Chữ nhật,6.0,15.0,Đất ở,Nhà phố liền kề. | - Địa chỉ: Hẻm 201 Ngô Chí ...,10.886863,106.723706,"[""https://file4.batdongsan.com.vn/crop/600x315..."


In [23]:
import sqlite3
import pandas as pd

# Connect to in-memory database (or use a file with "mydb.sqlite")
conn = sqlite3.connect(":memory:")

# Step 1: Create the table
conn.execute("""
CREATE TABLE provinces (
    name TEXT,
    code TEXT,
    status TEXT
);
""")

conn.execute("""
CREATE TABLE districts (
    name TEXT,
    code TEXT,
    province_code TEXT,
    status TEXT
);
""")

conn.execute("""
    CREATE TABLE streets (
    name TEXT,
    code TEXT,
    district_code TEXT,
    status TEXT
);
""")

# Step 2: Read and execute the insert script
with open("Dữ liệu địa giới hành chính\provinces_20250225_2.sql", "r", encoding="utf-8") as file_prov:
    conn.executescript(file_prov.read())

with open("Dữ liệu địa giới hành chính\districts_20250225_2.sql", "r", encoding="utf-8") as file_dis:
    dis_raw = file_dis.read()

with open("Dữ liệu địa giới hành chính/streets_20250225_2.sql", "r", encoding="utf-8") as file_stre:
    street_raw = file_stre.read()


dis_cleaned = dis_raw.replace("\\'", "''")
street_cleaned = street_raw.replace("\\'", "''")
lines = street_cleaned.splitlines()
for i, line in enumerate(lines):
    if '48695925' in line:
        print(f"Line {i}: {line}")
        print("Previous:", lines[i-1] if i > 0 else '')
        print("Next:", lines[i+1] if i + 1 < len(lines) else '')
        break
conn.executescript(dis_cleaned)
conn.executescript(street_cleaned)

# Step 3: Read into pandas
province = pd.read_sql_query("SELECT * FROM provinces", conn)
district = pd.read_sql_query("SELECT * FROM districts", conn)
streets = pd.read_sql_query("SELECT * FROM streets", conn)
conn.close()

Line 23960: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES ('\'', '48695925', '24120616', 'INACTIVE');
Previous: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES ('/', '48695846', '24120616', 'INACTIVE');
Next: INSERT INTO `streets` (`name`, `code`, `district_code`, `status`) VALUES ('Đường nội bộ', '48696053', '24120616', 'INACTIVE');


OperationalError: near "48695925": syntax error

In [ ]:
streets[streets['code'] == 48695925]

In [4]:
address_dict = {}

for i in range(len(province)):
    key = province['name'][i]
    code = province['code'][i]
    values = district[district['province_code'] == code]['name'].to_list()
    address_dict[key] = values

address_dict['Tỉnh Thanh Hóa']


['Huyện Hậu Lộc',
 'Huyện Cẩm Thủy',
 'Huyện Ngọc Lặc',
 'Huyện Như Thanh',
 'Huyện Lang Chánh',
 'Huyện Bá Thước',
 'Huyện Thường Xuân',
 'Huyện Như Xuân',
 'Huyện Quan Hóa',
 'Huyện Quan Sơn',
 'Huyện Mường Lát',
 'Thành phố Sầm Sơn',
 'Thị xã Nghi Sơn',
 'Huyện Hà Trung',
 'Huyện Nông Cống',
 'Huyện Triệu Sơn',
 'Huyện Thọ Xuân',
 'Huyện Thiệu Hóa',
 'Thành phố Thanh Hóa',
 'Thị Xã Bỉm Sơn',
 'Huyện Đông Sơn',
 'Huyện Hoằng Hóa',
 'Huyện Quảng Xương',
 'Huyện Nga Sơn',
 'Huyện Yên Định',
 'Huyện Vĩnh Lộc',
 'Huyện Thạch Thành']

In [5]:
import sqlite3
import pandas as pd

data = pd.read_csv('output\listing_details_cleaned.csv')

# Connect to in-memory database (or use a file with "mydb.sqlite")
conn = sqlite3.connect(":memory:")

# Step 1: Create the table
conn.execute("""
CREATE TABLE provinces (
    name TEXT,
    code TEXT,
    status TEXT
);
""")

conn.execute("""
CREATE TABLE districts (
    name TEXT,
    code TEXT,
    province_code TEXT,
    status TEXT
);
""")

# Step 2: Read and execute the insert script
with open("Dữ liệu địa giới hành chính\provinces_20250225_2.sql", "r", encoding="utf-8") as file_prov:
    conn.executescript(file_prov.read())

with open("Dữ liệu địa giới hành chính\districts_20250225_2.sql", "r", encoding="utf-8") as file_dis:
    dis_raw = file_dis.read()

dis_cleaned = dis_raw.replace("\\'", "''")
conn.executescript(dis_cleaned)

# Step 3: Read into pandas
province = pd.read_sql_query("SELECT * FROM provinces", conn)
district = pd.read_sql_query("SELECT * FROM districts", conn)
conn.close()

reverse_province = {
    prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
    for prov in province['name'].unique()
}
reverse_province['Bà Rịa Vũng Tàu'] = 'Tỉnh Bà Rịa - Vũng Tàu'

reverse_district = {
    dis.replace("Thành phố ", "").replace("Thành Phố ", "").replace("Huyện ", "").replace("Quận ", "").replace("Thị xã ", ""): dis
    for dis in district['name'].unique()
}

def change_province(row):
    prov = row['Tỉnh/Thành phố']
    new_prov = reverse_province[prov]
    #row['Tỉnh/Thành phố'] = new_prov

    return new_prov

def change_district(row):
    dis = row['Thành phố/Quận/Huyện/Thị xã']
    #new_dis = reverse_district[dis]
    if 'Thành phố' in dis or 'Thành Phố' in dis or 'Quận' in dis or 'Huyện' in dis or 'Thị xã' in dis:
        new_dis = dis
    elif dis == 'Plei Ku':
        new_dis = 'Thành phố Pleiku'
    elif dis == 'Tuy Hòa':
        new_dis = 'Thành phố Tuy Hòa'
    elif dis == 'Đảo Phú Quý':
        new_dis = 'Thành phố Phan Thiết'
    elif dis == 'Việt Yên':
        new_dis = 'Thị xã Việt Yên'
    else:
        new_dis = reverse_district[dis]
    return new_dis

new_province = data.apply(change_province, axis = 1)
new_district = data.apply(change_district, axis = 1)

data['Tỉnh/Thành phố'] = new_province
data['Thành phố/Quận/Huyện/Thị xã'] = new_district

In [6]:
data.dropna(subset=['Diện tích đất (m2)'], inplace = True)

def fill_length(row):
    return row['Diện tích đất (m2)'] / row['Kích thước mặt tiền (m)']

def fill_facade(row):
    return row['Diện tích đất (m2)'] / row['Kích thước chiều dài (m)']

def fill_both(row):
    return row['Diện tích đất (m2)']**0.5

mask_both = (data['Kích thước chiều dài (m)'].isna()) & (data['Kích thước mặt tiền (m)'].isna())
data.loc[mask_both, 'Kích thước chiều dài (m)'] = data.loc[mask_both].apply(fill_both, axis = 1)
data.loc[mask_both, 'Kích thước mặt tiền (m)']= data.loc[mask_both].apply(fill_both, axis = 1)
    
mask_length = data['Kích thước chiều dài (m)'].isna()
data.loc[mask_length, 'Kích thước chiều dài (m)'] = data.loc[mask_length].apply(fill_length, axis = 1)

mask_facade = data['Kích thước mặt tiền (m)'].isna()
data.loc[mask_facade, 'Kích thước mặt tiền (m)']= data.loc[mask_facade].apply(fill_facade, axis = 1)

In [7]:
data.dropna(subset=['Diện tích đất (m2)'], inplace = True)

def fill_length(row):
    if pd.isna(row['Kích thước mặt tiền (m)']):
        return (row['Diện tích đất (m2)'])**0.5
    else:
        return row['Diện tích đất (m2)'] / row['Kích thước mặt tiền (m)']

def fill_facade(row):
    if pd.isna(row['Kích thước chiều dài (m)']):
        return (row['Diện tích đất (m2)'])**0.5
    else:
        return row['Diện tích đất (m2)'] / row['Kích thước chiều dài (m)']
    
mask_length = data['Kích thước chiều dài (m)'].isna()
data.loc[mask_length, 'Kích thước chiều dài (m)'] = data.loc[mask_length].apply(fill_length, axis = 1)

mask_facade = data['Kích thước mặt tiền (m)'].isna()
data.loc[mask_facade, 'Kích thước mặt tiền (m)']= data.loc[mask_facade].apply(fill_facade, axis = 1)

In [17]:
pd.set_option('display.max_colwidth', None)
data[data['Diện tích đất (m2)'] < data['Kích thước mặt tiền (m)']][['Nguồn thông tin']].iloc[2]

Nguồn thông tin    https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-bieu-phuong-2-15/ban-dep-hem-p-2-q-5-ngang-4-dai-14-chi-8-ty-9-thuong-luong-pr43455725
Name: 2106, dtype: object

In [9]:
data[data['Diện tích đất (m2)'] < data['Kích thước chiều dài (m)']][['Diện tích đất (m2)', 'Kích thước chiều dài (m)']]

,Diện tích đất (m2),Kích thước chiều dài (m)
14,34.0,45.0
19,54.0,135.0
54,46.0,106.0
56,50.4,127.0
71,54.0,56.0
...,...,...
18859,45.0,85.0
18875,50.0,125.0
18877,39.0,40.0
18895,74.9,1862.0


In [10]:
raw_data = pd.read_csv('output\listing_details.csv')